# Detecting Payment Card Fraud

In this section, we'll look at a credit card fraud detection dataset, and build a binary classification model that can identify transactions as either fraudulent or valid, based on provided, *historical* data. In a [2016 study](https://nilsonreport.com/upload/content_promo/The_Nilson_Report_10-17-2016.pdf), it was estimated that credit card fraud was responsible for over 20 billion dollars in loss, worldwide. Accurately detecting cases of fraud is an ongoing area of research.

<img src=notebook_ims/fraud_detection.png width=50% />

### Labeled Data

The payment fraud data set (Dal Pozzolo et al. 2015) was downloaded from [Kaggle](https://www.kaggle.com/mlg-ulb/creditcardfraud/data). This has features and labels for thousands of credit card transactions, each of which is labeled as fraudulent or valid. In this notebook, we'd like to train a model based on the features of these transactions so that we can predict risky or fraudulent transactions in the future.

### Binary Classification

Since we have true labels to aim for, we'll take a **supervised learning** approach and train a binary classifier to sort data into one of our two transaction classes: fraudulent or valid.  We'll train a model on training data and see how well it generalizes on some test data.

The notebook will be broken down into a few steps:
* Loading and exploring the data
* Splitting the data into train/test sets
* Defining and training a LinearLearner, binary classifier
* Making improvements on the model
* Evaluating and comparing model test performance

### Making Improvements

A lot of this notebook will focus on making improvements, as discussed in [this SageMaker blog post](https://aws.amazon.com/blogs/machine-learning/train-faster-more-flexible-models-with-amazon-sagemaker-linear-learner/). Specifically, we'll address techniques for:

1. **Tuning a model's hyperparameters** and aiming for a specific metric, such as high recall or precision.
2. **Managing class imbalance**, which is when we have many more training examples in one class than another (in this case, many more valid transactions than fraudulent).

---

First, import the usual resources.

In [1]:
import io
import os
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd 

import boto3
import sagemaker
from sagemaker import get_execution_role

%matplotlib inline

I'm storing my **SageMaker variables** in the next cell:
* sagemaker_session: The SageMaker session we'll use for training models.
* bucket: The name of the default S3 bucket that we'll use for data storage.
* role: The IAM role that defines our data and model permissions.

In [2]:
# sagemaker session, role
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

# S3 bucket name
bucket = sagemaker_session.default_bucket()


## Loading and Exploring the Data

Next, I am loading the data and unzipping the data in the file `creditcardfraud.zip`. This directory will hold one csv file of all the transaction data, `creditcard.csv`.

As in previous notebooks, it's important to look at the distribution of data since this will inform how we develop a fraud detection model. We'll want to know: How many data points we have to work with, the number and type of features, and finally, the distribution of data over the classes (valid or fraudulent).

In [3]:
# only have to run once
!wget https://s3.amazonaws.com/video.udacity-data.com/topher/2019/January/5c534768_creditcardfraud/creditcardfraud.zip
!unzip creditcardfraud


--2021-03-30 00:51:11--  https://s3.amazonaws.com/video.udacity-data.com/topher/2019/January/5c534768_creditcardfraud/creditcardfraud.zip
Resolving s3.amazonaws.com (s3.amazonaws.com)... 54.231.114.36
Connecting to s3.amazonaws.com (s3.amazonaws.com)|54.231.114.36|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 69155632 (66M) [application/zip]
Saving to: ‘creditcardfraud.zip’

creditcardfraud.zip 100%[===================>]  65.95M  38.1MB/s    in 1.7s    

2021-03-30 00:51:13 (38.1 MB/s) - ‘creditcardfraud.zip’ saved [69155632/69155632]

Archive:  creditcardfraud.zip
  inflating: creditcard.csv          


In [4]:
# read in the csv file
local_data = 'creditcard.csv'

# print out some data
transaction_df = pd.read_csv(local_data)
print('Data shape (rows, cols): ', transaction_df.shape)
print()
transaction_df.head()

Data shape (rows, cols):  (284807, 31)



,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


### EXERCISE: Calculate the percentage of fraudulent data

Take a look at the distribution of this transaction data over the classes, valid and fraudulent. 

Complete the function `fraudulent_percentage`, below. Count up the number of data points in each class and calculate the *percentage* of the data points that are fraudulent.

In [5]:
# Calculate the fraction of data points that are fraudulent
def fraudulent_percentage(transaction_df):
    '''Calculate the fraction of all data points that have a 'Class' label of 1; fraudulent.
       :param transaction_df: Dataframe of all transaction data points; has a column 'Class'
       :return: A fractional percentage of fraudulent data points/all points
    '''
    
    # your code here
    
    fraud=transaction_df[transaction_df["Class"]==1]
    
    return fraud.shape[0]/transaction_df.shape[0]


Test out your code by calling your function and printing the result.

In [6]:
# call the function to calculate the fraud percentage
fraud_percentage = fraudulent_percentage(transaction_df)

print('Fraudulent percentage = ', fraud_percentage)
print('Total # of fraudulent pts: ', fraud_percentage*transaction_df.shape[0])
print('Out of (total) pts: ', transaction_df.shape[0])


Fraudulent percentage =  0.001727485630620034
Total # of fraudulent pts:  492.0
Out of (total) pts:  284807


### EXERCISE: Split into train/test datasets

In this example, we'll want to evaluate the performance of a fraud classifier; training it on some training data and testing it on *test data* that it did not see during the training process. So, we'll need to split the data into separate training and test sets.

Complete the `train_test_split` function, below. This function should:
* Shuffle the transaction data, randomly
* Split it into two sets according to the parameter `train_frac`
* Get train/test features and labels
* Return the tuples: (train_features, train_labels), (test_features, test_labels)

In [9]:
# split into train/test
def train_test_split(transaction_df, train_frac= 0.7, seed=1):
    '''Shuffle the data and randomly split into train and test sets;
       separate the class labels (the column in transaction_df) from the features.
       :param df: Dataframe of all credit card transaction data
       :param train_frac: The decimal fraction of data that should be training data
       :param seed: Random seed for shuffling and reproducibility, default = 1
       :return: Two tuples (in order): (train_features, train_labels), (test_features, test_labels)
       '''
    
    # shuffle and split the data
    np.random.seed(seed)
    l = list(transaction_df.index)
    np.random.shuffle(l)
    transaction_df = transaction_df.loc[l]
    
    
    size = transaction_df.shape[0]
    train = round(train_frac *size)
    test = size - train
    # shuffle and split the data
    
    train_features = transaction_df.iloc[:train].drop(["Class"],axis=1)
   
    train_labels = transaction_df.iloc[:train]["Class"]
    test_features = transaction_df.iloc[train:].drop(["Class"],axis=1)
    test_labels = transaction_df.iloc[train:]["Class"]
    
    return (train_features, train_labels), (test_features, test_labels)


### Test Cell

In the cells below, I'm creating the train/test data and checking to see that result makes sense. The tests below test that the above function splits the data into the expected number of points and that the labels are indeed, class labels (0, 1).

In [10]:
# get train/test data
(train_features, train_labels), (test_features, test_labels) = train_test_split(transaction_df, train_frac=0.7)

In [11]:
# manual test

# for a split of 0.7:0.3 there should be ~2.33x as many training as test pts
print('Training data pts: ', len(train_features))
print('Test data pts: ', len(test_features))
print()

# take a look at first item and see that it aligns with first row of data
print('First item: \n', train_features.iloc[0])
print('Label: ', train_labels.iloc[0])
print()

# test split
assert len(train_features) > 2.333*len(test_features), \
        'Unexpected number of train/test points for a train_frac=0.7'
# test labels
assert np.all(train_labels)== 0 or np.all(train_labels)== 1, \
        'Train labels should be 0s or 1s.'
assert np.all(test_labels)== 0 or np.all(test_labels)== 1, \
        'Test labels should be 0s or 1s.'
print('Tests passed!')

Training data pts:  199365
Test data pts:  85442

First item: 
 Time      119907.000000
V1            -0.611712
V2            -0.769705
V3            -0.149759
V4            -0.224877
V5             2.028577
V6            -2.019887
V7             0.292491
V8            -0.523020
V9             0.358468
V10            0.070050
V11           -0.854023
V12            0.547347
V13            0.616448
V14           -0.101785
V15           -0.608492
V16           -0.288559
V17           -0.606199
V18           -0.900746
V19           -0.201311
V20           -0.196039
V21           -0.075208
V22            0.045536
V23            0.380739
V24            0.023440
V25           -2.220686
V26           -0.201146
V27            0.066501
V28            0.221180
Amount         1.790000
Name: 169876, dtype: float64
Label:  0

Tests passed!


---
# Modeling

Now that you've uploaded your training data, it's time to define and train a model!

In this notebook, you'll define and train the SageMaker, built-in algorithm, [LinearLearner](https://sagemaker.readthedocs.io/en/stable/linear_learner.html). 

A LinearLearner has two main applications:
1. For regression tasks in which a linear line is fit to some data points, and you want to produce a predicted output value given some data point (example: predicting house prices given square area).
2. For binary classification, in which a line is separating two classes of data and effectively outputs labels; either 1 for data that falls above the line or 0 for points that fall on or below the line.

<img src='notebook_ims/linear_separator.png' width=50% />

In this case, we'll be using it for case 2, and we'll train it to separate data into our two classes: valid or fraudulent. 

### EXERCISE: Create a LinearLearner Estimator

You've had some practice instantiating built-in models in SageMaker. All estimators require some constructor arguments to be passed in. See if you can complete this task, instantiating a LinearLearner estimator, using only the [LinearLearner documentation](https://sagemaker.readthedocs.io/en/stable/linear_learner.html) as a resource. This takes in a lot of arguments, but not all are required. My suggestion is to start with a simple model, utilizing default values where applicable. Later, we will discuss some specific hyperparameters and their use cases.

#### Instance Types

It is suggested that you use instances that are available in the free tier of usage: `'ml.c4.xlarge'` for training and `'ml.t2.medium'` for deployment.

In [12]:
# import LinearLearner
from sagemaker import LinearLearner

# instantiate LinearLearner
Linear_model=LinearLearner(role=role,
                           instance_count=1,
                           instance_type='ml.c4.xlarge',
                           predictor_type='binary_classifier')

### EXERCISE: Convert data into a RecordSet format

Next, prepare the data for a built-in model by converting the train features and labels into numpy array's of float values. Then you can use the [record_set function](https://sagemaker.readthedocs.io/en/stable/linear_learner.html#sagemaker.LinearLearner.record_set) to format the data as a RecordSet and prepare it for training!

In [19]:
# create RecordSet of training data
formatted_train_data = Linear_model.record_set(train_features.to_numpy(dtype=np.float32),train_labels.to_numpy(dtype=np.float32), channel='train')

### EXERCISE: Train the Estimator

After instantiating your estimator, train it with a call to `.fit()`, passing in the formatted training data.

In [20]:
%%time 
# train the estimator on formatted training data
Linear_model.fit([formatted_train_data], wait=True, logs='All')

Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.


2021-03-30 01:00:46 Starting - Starting the training job...
2021-03-30 01:00:49 Starting - Launching requested ML instancesProfilerReport-1617066045: InProgress
.........
2021-03-30 01:02:34 Starting - Preparing the instances for training......
2021-03-30 01:03:34 Downloading - Downloading input data...
2021-03-30 01:04:14 Training - Downloading the training image.Docker entrypoint called with argument(s): train
Running default environment configuration script
[03/30/2021 01:04:25 INFO 139970648151872] Reading default configuration from /opt/amazon/lib/python3.7/site-packages/algorithm/resources/default-input.json: {'mini_batch_size': '1000', 'epochs': '15', 'feature_dim': 'auto', 'use_bias': 'true', 'binary_classifier_model_selection_criteria': 'accuracy', 'f_beta': '1.0', 'target_recall': '0.8', 'target_precision': '0.8', 'num_models': 'auto', 'num_calibration_samples': '10000000', 'init_method': 'uniform', 'init_scale': '0.07', 'init_sigma': '0.01', 'init_bias': '0.0', 'optimizer': 

[2021-03-30 01:04:46.859] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 7, "duration": 6631, "num_examples": 200, "num_bytes": 33493320}
#metrics {"StartTime": 1617066286.8591208, "EndTime": 1617066286.8592095, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 2, "model": 0}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.01033092862038157, "count": 1, "min": 0.01033092862038157, "max": 0.01033092862038157}}}

#metrics {"StartTime": 1617066286.859298, "EndTime": 1617066286.8593209, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 2, "model": 1}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.008765228142091377, "count": 1, "min": 0.008765228142091377, "max": 0.008765228142091377}}}

#metrics {"StartTime": 1617066286.8593833, "EndTime": 1617066286.859403, "Dimensions": {"Algorithm": "Linear Lea

[2021-03-30 01:04:53.383] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 9, "duration": 6511, "num_examples": 200, "num_bytes": 33493320}
#metrics {"StartTime": 1617066293.383204, "EndTime": 1617066293.3832552, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 3, "model": 0}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.008034130625988371, "count": 1, "min": 0.008034130625988371, "max": 0.008034130625988371}}}

#metrics {"StartTime": 1617066293.3833215, "EndTime": 1617066293.3833408, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 3, "model": 1}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.006699231853437184, "count": 1, "min": 0.006699231853437184, "max": 0.006699231853437184}}}

#metrics {"StartTime": 1617066293.3833919, "EndTime": 1617066293.3834076, "Dimensions": {"Algorithm": "Linear

[2021-03-30 01:05:06.720] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 13, "duration": 7310, "num_examples": 200, "num_bytes": 33493320}
#metrics {"StartTime": 1617066306.7203536, "EndTime": 1617066306.7205806, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 5, "model": 0}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.006276343730226833, "count": 1, "min": 0.006276343730226833, "max": 0.006276343730226833}}}

#metrics {"StartTime": 1617066306.720681, "EndTime": 1617066306.720701, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 5, "model": 1}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.005225318890720157, "count": 1, "min": 0.005225318890720157, "max": 0.005225318890720157}}}

#metrics {"StartTime": 1617066306.7207537, "EndTime": 1617066306.7207656, "Dimensions": {"Algorithm": "Linear

...

[2021-03-30 01:05:38.946] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 23, "duration": 6601, "num_examples": 200, "num_bytes": 33493320}
#metrics {"StartTime": 1617066338.9469147, "EndTime": 1617066338.9469924, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 10, "model": 0}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.005129939451888578, "count": 1, "min": 0.005129939451888578, "max": 0.005129939451888578}}}

#metrics {"StartTime": 1617066338.9470623, "EndTime": 1617066338.9470816, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 10, "model": 1}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.004400380551215991, "count": 1, "min": 0.004400380551215991, "max": 0.004400380551215991}}}

#metrics {"StartTime": 1617066338.9471369, "EndTime": 1617066338.9471555, "Dimensions": {"Algorithm": "Li

[2021-03-30 01:05:45.656] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 25, "duration": 6698, "num_examples": 200, "num_bytes": 33493320}
#metrics {"StartTime": 1617066345.6567478, "EndTime": 1617066345.656833, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 11, "model": 0}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.005040883766945882, "count": 1, "min": 0.005040883766945882, "max": 0.005040883766945882}}}

#metrics {"StartTime": 1617066345.6569564, "EndTime": 1617066345.6569803, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 11, "model": 1}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.0043473425067249855, "count": 1, "min": 0.0043473425067249855, "max": 0.0043473425067249855}}}

#metrics {"StartTime": 1617066345.6570318, "EndTime": 1617066345.6570492, "Dimensions": {"Algorithm": "

[2021-03-30 01:05:58.722] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 29, "duration": 6496, "num_examples": 200, "num_bytes": 33493320}
#metrics {"StartTime": 1617066358.722066, "EndTime": 1617066358.7221396, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 13, "model": 0}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.004912319250442275, "count": 1, "min": 0.004912319250442275, "max": 0.004912319250442275}}}

#metrics {"StartTime": 1617066358.722215, "EndTime": 1617066358.7222347, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 13, "model": 1}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.004274392569484423, "count": 1, "min": 0.004274392569484423, "max": 0.004274392569484423}}}

#metrics {"StartTime": 1617066358.7222855, "EndTime": 1617066358.7223017, "Dimensions": {"Algorithm": "Line


2021-03-30 01:06:17 Uploading - Uploading generated training model
2021-03-30 01:06:17 Completed - Training job completed
[2021-03-30 01:06:05.500] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 31, "duration": 6768, "num_examples": 200, "num_bytes": 33493320}
#metrics {"StartTime": 1617066365.5002494, "EndTime": 1617066365.5004985, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 14, "model": 0}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.004865119796901492, "count": 1, "min": 0.004865119796901492, "max": 0.004865119796901492}}}

#metrics {"StartTime": 1617066365.5006113, "EndTime": 1617066365.5006313, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 14, "model": 1}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.004249175122035808, "count": 1, "min": 0.004249175122035808, "max": 0.00424

Training seconds: 170
Billable seconds: 170
CPU times: user 917 ms, sys: 72.7 ms, total: 989 ms
Wall time: 5min 44s


### EXERCISE: Deploy the trained model

Deploy your model to create a predictor. We'll use this to make predictions on our test data and evaluate the model.

In [21]:
%%time 
# deploy and create a predictor
linear_predictor = Linear_model.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.


---------------!CPU times: user 277 ms, sys: 13 ms, total: 290 ms
Wall time: 7min 33s


---
# Evaluating Your Model

Once your model is deployed, you can see how it performs when applied to the test data.

According to the deployed [predictor documentation](https://sagemaker.readthedocs.io/en/stable/linear_learner.html#sagemaker.LinearLearnerPredictor), this predictor expects an `ndarray` of input features and returns a list of Records.
> "The prediction is stored in the "predicted_label" key of the `Record.label` field."

Let's first test our model on just one test point, to see the resulting list.

In [26]:
# test one prediction
test_x_np = test_features.to_numpy(dtype=np.float32)
print(test_x_np)



[[ 3.4487000e+04  1.0271744e+00 -7.3948252e-01 ... -4.2809457e-02
   5.5417437e-03  5.1000000e+01]
 [ 5.4868000e+04 -3.8019481e+00  5.2335935e+00 ... -2.6913854e-01
  -3.8614231e-01  1.2900000e+00]
 [ 6.7478000e+04  1.0078337e+00 -1.6314390e-01 ... -4.9463596e-02
   1.5755044e-02  8.6529999e+01]
 ...
 [ 5.5095000e+04 -1.1499633e+00  1.6964617e+00 ... -1.6204654e-01
   3.1192789e-02  7.5799999e+00]
 [ 1.6272800e+05  1.7545537e+00 -6.9939828e-01 ... -5.8529084e-03
  -1.9622076e-02  9.4800003e+01]
 [ 7.8576000e+04 -7.0763499e-01  4.9330226e-01 ... -3.8558862e-01
  -2.0558894e-01  5.7799999e+01]]


In [27]:
result = linear_predictor.predict(test_x_np[0])

print(result)

[label {
  key: "predicted_label"
  value {
    float32_tensor {
      values: 0.0
    }
  }
}
label {
  key: "score"
  value {
    float32_tensor {
      values: 0.0004113880277145654
    }
  }
}
]


### Helper function for evaluation


The provided function below, takes in a deployed predictor, some test features and labels, and returns a dictionary of metrics; calculating false negatives and positives as well as recall, precision, and accuracy.

In [28]:
# code to evaluate the endpoint on test data
# returns a variety of model metrics
def evaluate(predictor, test_features, test_labels, verbose=True):
    """
    Evaluate a model on a test set given the prediction endpoint.  
    Return binary classification metrics.
    :param predictor: A prediction endpoint
    :param test_features: Test features
    :param test_labels: Class labels for test data
    :param verbose: If True, prints a table of all performance metrics
    :return: A dictionary of performance metrics.
    """
    
    # We have a lot of test data, so we'll split it into batches of 100
    # split the test data set into batches and evaluate using prediction endpoint    
    prediction_batches = [predictor.predict(batch) for batch in np.array_split(test_features, 100)]
    
    # LinearLearner produces a `predicted_label` for each data point in a batch
    # get the 'predicted_label' for every point in a batch
    test_preds = np.concatenate([np.array([x.label['predicted_label'].float32_tensor.values[0] for x in batch]) 
                                 for batch in prediction_batches])
    
    # calculate true positives, false positives, true negatives, false negatives
    tp = np.logical_and(test_labels, test_preds).sum()
    fp = np.logical_and(1-test_labels, test_preds).sum()
    tn = np.logical_and(1-test_labels, 1-test_preds).sum()
    fn = np.logical_and(test_labels, 1-test_preds).sum()
    
    # calculate binary classification metrics
    recall = tp / (tp + fn)
    precision = tp / (tp + fp)
    accuracy = (tp + tn) / (tp + fp + tn + fn)
    
    # printing a table of metrics
    if verbose:
        print(pd.crosstab(test_labels, test_preds, rownames=['actual (row)'], colnames=['prediction (col)']))
        print("\n{:<11} {:.3f}".format('Recall:', recall))
        print("{:<11} {:.3f}".format('Precision:', precision))
        print("{:<11} {:.3f}".format('Accuracy:', accuracy))
        print()
        
    return {'TP': tp, 'FP': fp, 'FN': fn, 'TN': tn, 
            'Precision': precision, 'Recall': recall, 'Accuracy': accuracy}


### Test Results

The cell below runs the `evaluate` function. 

The code assumes that you have a defined `predictor` and `test_features` and `test_labels` from previously-run cells.

In [30]:
print('Metrics for simple, LinearLearner.\n')

# get metrics for linear predictor
metrics = evaluate(linear_predictor, 
                   test_features.to_numpy(dtype=np.float32), 
                   test_labels, 
                   verbose=True) # verbose means we'll print out the metrics


Metrics for simple, LinearLearner.

prediction (col)    0.0  1.0
actual (row)                
0                 85268   33
1                    32  109

Recall:     0.773
Precision:  0.768
Accuracy:   0.999



## Delete the Endpoint

I've added a convenience function to delete prediction endpoints after we're done with them. And if you're done evaluating the model, you should delete your model endpoint!

In [31]:
# Deletes a precictor.endpoint
def delete_endpoint(predictor):
        try:
            boto3.client('sagemaker').delete_endpoint(EndpointName=predictor.endpoint)
            print('Deleted {}'.format(predictor.endpoint))
        except:
            print('Already deleted: {}'.format(predictor.endpoint))

In [32]:
# delete the predictor endpoint 
delete_endpoint(linear_predictor)

The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


Deleted linear-learner-2021-03-30-01-06-30-033


---

# Model Improvements

The default LinearLearner got a high accuracy, but still classified fraudulent and valid data points incorrectly. Specifically classifying more than 30 points as false negatives (incorrectly labeled, fraudulent transactions), and a little over 30 points as false positives (incorrectly labeled, valid transactions). Let's think about what, during training, could cause this behavior and what we could improve.

**1. Model optimization**
* If we imagine that we are designing this model for use in a bank application, we know that users do *not* want any valid transactions to be categorized as fraudulent. That is, we want to have as few **false positives** (0s classified as 1s) as possible. 
* On the other hand, if our bank manager asks for an application that will catch almost *all* cases of fraud, even if it means a higher number of false positives, then we'd want as few **false negatives** as possible.
* To train according to specific product demands and goals, we do not want to optimize for accuracy only. Instead, we want to optimize for a metric that can help us decrease the number of false positives or negatives. 

<img src='notebook_ims/precision_recall.png' width=40% />
     
In this notebook, we'll look at different cases for tuning a model and make an optimization decision, accordingly.

**2. Imbalanced training data**
* At the start of this notebook, we saw that only about 0.17% of the training data was labeled as fraudulent. So, even if a model labels **all** of our data as valid, it will still have a high accuracy. 
* This may result in some overfitting towards valid data, which accounts for some **false negatives**; cases in which fraudulent data (1) is incorrectly characterized as valid (0).

So, let's address these issues in order; first, tuning our model and optimizing for a specific metric during training, and second, accounting for class imbalance in the training set. 


## Improvement: Model Tuning

Optimizing according to a specific metric is called **model tuning**, and SageMaker provides a number of ways to automatically tune a model.


### Create a LinearLearner and tune for higher precision 

**Scenario:**
* A bank has asked you to build a model that detects cases of fraud with an accuracy of about 85%. 

In this case, we want to build a model that has as many true positives and as few false negatives, as possible. This corresponds to a model with a high **recall**: true positives / (true positives + false negatives). 

To aim for a specific metric, LinearLearner offers the hyperparameter `binary_classifier_model_selection_criteria`, which is the model evaluation criteria for the training dataset. A reference to this parameter is in [LinearLearner's documentation](https://sagemaker.readthedocs.io/en/stable/linear_learner.html#sagemaker.LinearLearner). We'll also have to further specify the exact value we want to aim for; read more about the details of the parameters, [here](https://docs.aws.amazon.com/sagemaker/latest/dg/ll_hyperparameters.html).

I will assume that performance on a training set will be within about 5% of the performance on a test set. So, for a recall of about 85%, I'll aim for a bit higher, 90%.

In [34]:
prefix = 'FraudDetection'

output_path='s3://{}/{}/'.format(bucket, prefix)

print('Training artifacts will be uploaded to: {}'.format(output_path))

Training artifacts will be uploaded to: s3://sagemaker-us-east-1-534407698314/FraudDetection/


In [35]:
# instantiate a LinearLearner
# tune the model for a higher recall
linear_recall = LinearLearner(role=role,
                              train_instance_count=1, 
                              train_instance_type='ml.c4.xlarge',
                              predictor_type='binary_classifier',
                              output_path=output_path,
                              sagemaker_session=sagemaker_session,
                              epochs=15,
                              binary_classifier_model_selection_criteria='precision_at_target_recall', # target recall
                              target_recall=0.9) # 90% recall


train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


### Train the tuned estimator

Fit the new, tuned estimator on the formatted training data.

In [36]:
%%time 
# train the estimator on formatted training data
linear_recall.fit(formatted_train_data)

Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.
Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.


2021-03-30 01:24:36 Starting - Starting the training job...
2021-03-30 01:24:59 Starting - Launching requested ML instancesProfilerReport-1617067476: InProgress
......
2021-03-30 01:26:00 Starting - Preparing the instances for training.........
2021-03-30 01:27:30 Downloading - Downloading input data......
2021-03-30 01:28:35 Training - Training image download completed. Training in progress..Docker entrypoint called with argument(s): train
Running default environment configuration script
[03/30/2021 01:28:41 INFO 140114959800128] Reading default configuration from /opt/amazon/lib/python3.7/site-packages/algorithm/resources/default-input.json: {'mini_batch_size': '1000', 'epochs': '15', 'feature_dim': 'auto', 'use_bias': 'true', 'binary_classifier_model_selection_criteria': 'accuracy', 'f_beta': '1.0', 'target_recall': '0.8', 'target_precision': '0.8', 'num_models': 'auto', 'num_calibration_samples': '10000000', 'init_method': 'uniform', 'init_scale': '0.07', 'init_sigma': '0.01', 'ini

[2021-03-30 01:28:55.991] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 5, "duration": 7011, "num_examples": 200, "num_bytes": 33493320}
#metrics {"StartTime": 1617067735.9913573, "EndTime": 1617067735.9914303, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 1, "model": 0}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.01697396406815879, "count": 1, "min": 0.01697396406815879, "max": 0.01697396406815879}}}

#metrics {"StartTime": 1617067735.9915133, "EndTime": 1617067735.9915285, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 1, "model": 1}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.014988558927373071, "count": 1, "min": 0.014988558927373071, "max": 0.014988558927373071}}}

#metrics {"StartTime": 1617067735.9915695, "EndTime": 1617067735.9915807, "Dimensions": {"Algorithm": "Linear L

[2021-03-30 01:29:09.860] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 9, "duration": 7074, "num_examples": 200, "num_bytes": 33493320}
#metrics {"StartTime": 1617067749.8603694, "EndTime": 1617067749.86044, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 3, "model": 0}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.008034130625988371, "count": 1, "min": 0.008034130625988371, "max": 0.008034130625988371}}}

#metrics {"StartTime": 1617067749.8605328, "EndTime": 1617067749.8605535, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 3, "model": 1}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.006699231853437184, "count": 1, "min": 0.006699231853437184, "max": 0.006699231853437184}}}

#metrics {"StartTime": 1617067749.8606055, "EndTime": 1617067749.8606222, "Dimensions": {"Algorithm": "Linear 

[2021-03-30 01:29:16.600] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 11, "duration": 6728, "num_examples": 200, "num_bytes": 33493320}
#metrics {"StartTime": 1617067756.6005533, "EndTime": 1617067756.6006317, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 4, "model": 0}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.006916961988612036, "count": 1, "min": 0.006916961988612036, "max": 0.006916961988612036}}}

#metrics {"StartTime": 1617067756.6007278, "EndTime": 1617067756.6007485, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 4, "model": 1}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.005745243255816513, "count": 1, "min": 0.005745243255816513, "max": 0.005745243255816513}}}

#metrics {"StartTime": 1617067756.6008072, "EndTime": 1617067756.6008248, "Dimensions": {"Algorithm": "Line

...

[2021-03-30 01:29:48.334] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 21, "duration": 6098, "num_examples": 200, "num_bytes": 33493320}
#metrics {"StartTime": 1617067788.3345933, "EndTime": 1617067788.334678, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 9, "model": 0}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.005243560697565127, "count": 1, "min": 0.005243560697565127, "max": 0.005243560697565127}}}

#metrics {"StartTime": 1617067788.3347633, "EndTime": 1617067788.3347843, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 9, "model": 1}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.004472267503714442, "count": 1, "min": 0.004472267503714442, "max": 0.004472267503714442}}}

#metrics {"StartTime": 1617067788.3348415, "EndTime": 1617067788.3348596, "Dimensions": {"Algorithm": "Linea

[2021-03-30 01:29:54.333] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 23, "duration": 5988, "num_examples": 200, "num_bytes": 33493320}
#metrics {"StartTime": 1617067794.3335872, "EndTime": 1617067794.3336704, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 10, "model": 0}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.005129939451888578, "count": 1, "min": 0.005129939451888578, "max": 0.005129939451888578}}}

#metrics {"StartTime": 1617067794.3337693, "EndTime": 1617067794.3337846, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 10, "model": 1}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.004400380551215991, "count": 1, "min": 0.004400380551215991, "max": 0.004400380551215991}}}

#metrics {"StartTime": 1617067794.3338456, "EndTime": 1617067794.3338633, "Dimensions": {"Algorithm": "Li

[2021-03-30 01:30:06.823] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 27, "duration": 6626, "num_examples": 200, "num_bytes": 33493320}
#metrics {"StartTime": 1617067806.823572, "EndTime": 1617067806.8236763, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 12, "model": 0}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.004969843113841723, "count": 1, "min": 0.004969843113841723, "max": 0.004969843113841723}}}

#metrics {"StartTime": 1617067806.8237839, "EndTime": 1617067806.8238046, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 12, "model": 1}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.0043063429063888055, "count": 1, "min": 0.0043063429063888055, "max": 0.0043063429063888055}}}

#metrics {"StartTime": 1617067806.8238702, "EndTime": 1617067806.8238878, "Dimensions": {"Algorithm": "

[2021-03-30 01:30:20.165] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 31, "duration": 6526, "num_examples": 200, "num_bytes": 33493320}
#metrics {"StartTime": 1617067820.1654065, "EndTime": 1617067820.1656451, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 14, "model": 0}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.004865119796901492, "count": 1, "min": 0.004865119796901492, "max": 0.004865119796901492}}}

#metrics {"StartTime": 1617067820.1657348, "EndTime": 1617067820.1657536, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 14, "model": 1}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.004249175122035808, "count": 1, "min": 0.004249175122035808, "max": 0.004249175122035808}}}

#metrics {"StartTime": 1617067820.1657991, "EndTime": 1617067820.16581, "Dimensions": {"Algorithm": "Line


2021-03-30 01:30:31 Uploading - Uploading generated training model
2021-03-30 01:30:31 Completed - Training job completed
Training seconds: 181
Billable seconds: 181
CPU times: user 917 ms, sys: 50.9 ms, total: 968 ms
Wall time: 6min 14s


### Deploy and evaluate the tuned estimator

Deploy the tuned predictor and evaluate it.

We hypothesized that a tuned model, optimized for a higher recall, would have fewer false negatives (fraudulent transactions incorrectly labeled as valid); did the number of false negatives get reduced after tuning the model?

In [37]:
%%time 
# deploy and create a predictor
recall_predictor = linear_recall.deploy(initial_instance_count=1, instance_type='ml.t2.medium')

Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.


-------------------!CPU times: user 335 ms, sys: 17.7 ms, total: 352 ms
Wall time: 9min 32s


In [39]:
print('Metrics for tuned (recall), LinearLearner.\n')

# get metrics for tuned predictor
metrics = evaluate(recall_predictor, 
                   test_features.to_numpy(dtype=np.float32), 
                   test_labels, 
                   verbose=True)

Metrics for tuned (recall), LinearLearner.

prediction (col)    0.0   1.0
actual (row)                 
0                 81913  3388
1                    10   131

Recall:     0.929
Precision:  0.037
Accuracy:   0.960



## Delete the endpoint 

As always, when you're done evaluating a model, you should delete the endpoint. Below, I'm using the `delete_endpoint` helper function I defined earlier.

In [40]:
# delete the predictor endpoint 
delete_endpoint(recall_predictor)

The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


Deleted linear-learner-2021-03-30-01-30-51-052


---
## Improvement: Managing Class Imbalance

We have a model that is tuned to get a higher recall, which aims to reduce the number of false negatives. Earlier, we discussed how class imbalance may actually bias our model towards predicting that all transactions are valid, resulting in higher false negatives and true negatives. It stands to reason that this model could be further improved if we account for this imbalance.

To account for class imbalance during training of a binary classifier, LinearLearner offers the hyperparameter, `positive_example_weight_mult`, which is the weight assigned to positive (1, fraudulent) examples when training a binary classifier. The weight of negative examples (0, valid) is fixed at 1. 

### EXERCISE: Create a LinearLearner with a `positive_example_weight_mult` parameter

In **addition** to tuning a model for higher recall (you may use `linear_recall` as a starting point), you should *add* a parameter that helps account for class imbalance. From the [hyperparameter documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/ll_hyperparameters.html) on `positive_example_weight_mult`, it reads:
> "If you want the algorithm to choose a weight so that errors in classifying negative vs. positive examples have equal impact on training loss, specify `balanced`."

You could also put in a specific float value, in which case you'd want to weight positive examples more heavily than negative examples, since there are fewer of them.

In [41]:
# instantiate a LinearLearner

# include params for tuning for higher recall
# *and* account for class imbalance in training data
linear_balanced = linear_recall = LinearLearner(role=role,
                              train_instance_count=1, 
                              train_instance_type='ml.c4.xlarge',
                              predictor_type='binary_classifier',
                              output_path=output_path,
                              sagemaker_session=sagemaker_session,
                              epochs=15,
                              binary_classifier_model_selection_criteria='precision_at_target_recall', # target recall
                              target_recall=0.9, # 90% recall
                              positive_example_weight_mult='balanced') 


train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


### EXERCISE: Train the balanced estimator

Fit the new, balanced estimator on the formatted training data.

In [42]:
%%time 
# train the estimator on formatted training data
linear_balanced.fit(formatted_train_data)

Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.
Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.


2021-03-30 01:44:36 Starting - Starting the training job...
2021-03-30 01:44:59 Starting - Launching requested ML instancesProfilerReport-1617068675: InProgress
......
2021-03-30 01:45:59 Starting - Preparing the instances for training.........
2021-03-30 01:47:30 Downloading - Downloading input data...
2021-03-30 01:48:06 Training - Training image download completed. Training in progress..Docker entrypoint called with argument(s): train
Running default environment configuration script
[03/30/2021 01:48:13 INFO 140219497084736] Reading default configuration from /opt/amazon/lib/python3.7/site-packages/algorithm/resources/default-input.json: {'mini_batch_size': '1000', 'epochs': '15', 'feature_dim': 'auto', 'use_bias': 'true', 'binary_classifier_model_selection_criteria': 'accuracy', 'f_beta': '1.0', 'target_recall': '0.8', 'target_precision': '0.8', 'num_models': 'auto', 'num_calibration_samples': '10000000', 'init_method': 'uniform', 'init_scale': '0.07', 'init_sigma': '0.01', 'init_b

[2021-03-30 01:48:35.710] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 7, "duration": 6845, "num_examples": 200, "num_bytes": 33493320}
#metrics {"StartTime": 1617068915.7107255, "EndTime": 1617068915.7108085, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 2, "model": 0}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.3963692446761395, "count": 1, "min": 0.3963692446761395, "max": 0.3963692446761395}}}

#metrics {"StartTime": 1617068915.710883, "EndTime": 1617068915.7109003, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 2, "model": 1}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.37658528819635284, "count": 1, "min": 0.37658528819635284, "max": 0.37658528819635284}}}

#metrics {"StartTime": 1617068915.7109566, "EndTime": 1617068915.7109742, "Dimensions": {"Algorithm"

[2021-03-30 01:48:43.065] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 9, "duration": 7343, "num_examples": 200, "num_bytes": 33493320}
#metrics {"StartTime": 1617068923.0659695, "EndTime": 1617068923.0662308, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 3, "model": 0}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.3764211316132665, "count": 1, "min": 0.3764211316132665, "max": 0.3764211316132665}}}

#metrics {"StartTime": 1617068923.0663486, "EndTime": 1617068923.0663662, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 3, "model": 1}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.36157515816233265, "count": 1, "min": 0.36157515816233265, "max": 0.36157515816233265}}}

#metrics {"StartTime": 1617068923.0664172, "EndTime": 1617068923.0664291, "Dimensions": {"Algorithm

[2021-03-30 01:48:57.418] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 13, "duration": 7355, "num_examples": 200, "num_bytes": 33493320}
#metrics {"StartTime": 1617068937.4183362, "EndTime": 1617068937.4184294, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 5, "model": 0}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.35806176941838097, "count": 1, "min": 0.35806176941838097, "max": 0.35806176941838097}}}

#metrics {"StartTime": 1617068937.4185283, "EndTime": 1617068937.4185479, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 5, "model": 1}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.34855324841983354, "count": 1, "min": 0.34855324841983354, "max": 0.34855324841983354}}}

#metrics {"StartTime": 1617068937.4186146, "EndTime": 1617068937.4186335, "Dimensions": {"Algor

...

[2021-03-30 01:49:25.961] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 21, "duration": 6862, "num_examples": 200, "num_bytes": 33493320}
#metrics {"StartTime": 1617068965.9617531, "EndTime": 1617068965.9618378, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 9, "model": 0}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.34497470242294237, "count": 1, "min": 0.34497470242294237, "max": 0.34497470242294237}}}

#metrics {"StartTime": 1617068965.9619145, "EndTime": 1617068965.9619327, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 9, "model": 1}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.33982735768993894, "count": 1, "min": 0.33982735768993894, "max": 0.33982735768993894}}}

#metrics {"StartTime": 1617068965.9619906, "EndTime": 1617068965.9620085, "Dimensions": {"Algor

[2021-03-30 01:49:39.424] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 25, "duration": 6849, "num_examples": 200, "num_bytes": 33493320}
#metrics {"StartTime": 1617068979.4249492, "EndTime": 1617068979.4250395, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 11, "model": 0}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.3421441708282011, "count": 1, "min": 0.3421441708282011, "max": 0.3421441708282011}}}

#metrics {"StartTime": 1617068979.425136, "EndTime": 1617068979.425156, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 11, "model": 1}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.3377189590607456, "count": 1, "min": 0.3377189590607456, "max": 0.3377189590607456}}}

#metrics {"StartTime": 1617068979.4252105, "EndTime": 1617068979.4252222, "Dimensions": {"Algorithm":

[2021-03-30 01:49:46.498] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 27, "duration": 7062, "num_examples": 200, "num_bytes": 33493320}
#metrics {"StartTime": 1617068986.498099, "EndTime": 1617068986.4981945, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 12, "model": 0}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.3411415676327806, "count": 1, "min": 0.3411415676327806, "max": 0.3411415676327806}}}

#metrics {"StartTime": 1617068986.4982853, "EndTime": 1617068986.498304, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 12, "model": 1}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.3368815051898285, "count": 1, "min": 0.3368815051898285, "max": 0.3368815051898285}}}

#metrics {"StartTime": 1617068986.4983795, "EndTime": 1617068986.498399, "Dimensions": {"Algorithm": 


2021-03-30 01:50:04 Uploading - Uploading generated training model[2021-03-30 01:50:00.330] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 31, "duration": 7231, "num_examples": 200, "num_bytes": 33493320}
#metrics {"StartTime": 1617069000.3310127, "EndTime": 1617069000.3310945, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 14, "model": 0}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.3396845555137749, "count": 1, "min": 0.3396845555137749, "max": 0.3396845555137749}}}

#metrics {"StartTime": 1617069000.3311753, "EndTime": 1617069000.3311913, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 14, "model": 1}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.33545820479656585, "count": 1, "min": 0.33545820479656585, "max": 0.33545820479656585}}}

#metrics {"StartTime": 161706


2021-03-30 01:50:20 Completed - Training job completed
ProfilerReport-1617068675: NoIssuesFound
Training seconds: 163
Billable seconds: 163
CPU times: user 941 ms, sys: 37.9 ms, total: 979 ms
Wall time: 6min 14s


### EXERCISE: Deploy and evaluate the balanced estimator

Deploy the balanced predictor and evaluate it. Do the results match with your expectations?

In [43]:
%%time 
# deploy and create a predictor
balanced_predictor = linear_balanced.deploy(initial_instance_count=1, instance_type='ml.t2.medium')

Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.


---------------------!CPU times: user 359 ms, sys: 17.4 ms, total: 376 ms
Wall time: 10min 32s


In [44]:
print('Metrics for balanced, LinearLearner.\n')

# get metrics for balanced predictor
metrics = evaluate(balanced_predictor, 
                   test_features.to_numpy(dtype=np.float32), 
                   test_labels, 
                   verbose=True)

Metrics for balanced, LinearLearner.

prediction (col)    0.0   1.0
actual (row)                 
0                 84276  1025
1                    12   129

Recall:     0.915
Precision:  0.112
Accuracy:   0.988



## Delete the endpoint 

When you're done evaluating a model, you should delete the endpoint.

In [45]:
# delete the predictor endpoint 
delete_endpoint(balanced_predictor)

The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


Deleted linear-learner-2021-03-30-01-50-50-827


A note on metric variability: 

The above model is tuned for the best possible precision with recall fixed at about 90%. The recall is fixed at 90% during training, but may vary when we apply our trained model to a test set of data.

---
## Model Design

Now that you've seen how to tune and balance a LinearLearner. Create, train and deploy your own model. This exercise is meant to be more open-ended, so that you get practice with the steps involved in designing a model and deploying it.

### EXERCISE: Train and deploy a LinearLearner with appropriate hyperparameters, according to the given scenario

**Scenario:**
* A bank has asked you to build a model that optimizes for a good user experience; users should only ever have up to about 15% of their valid transactions flagged as fraudulent.

This requires that you make a design decision: Given the above scenario, what metric (and value) should you aim for during training?

You may assume that performance on a training set will be within about 5-10% of the performance on a test set. For example, if you get 80% on a training set, you can assume that you'll get between about 70-90% accuracy on a test set.

Your final model should account for class imbalance and be appropriately tuned. 

In [46]:
%%time
# instantiate and train a LinearLearner

# include params for tuning for higher precision
# *and* account for class imbalance in training data
# instantiate a LinearLearner

# include params for tuning for higher recall
# *and* account for class imbalance in training data
linear_modeldesign= LinearLearner(role=role,
                              train_instance_count=1, 
                              train_instance_type='ml.c4.xlarge',
                              predictor_type='binary_classifier',
                              output_path=output_path,
                              sagemaker_session=sagemaker_session,
                              epochs=15,
                              binary_classifier_model_selection_criteria='recall_at_target_precision', # target precision
                              target_precision=0.85, # 85% precision
                              positive_example_weight_mult='balanced') 

train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


CPU times: user 4.12 ms, sys: 1 µs, total: 4.12 ms
Wall time: 3.77 ms


In [47]:
%%time 
# train the estimator on formatted training data
linear_modeldesign.fit(formatted_train_data)

Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.
Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.


2021-03-30 02:11:03 Starting - Starting the training job...
2021-03-30 02:11:27 Starting - Launching requested ML instancesProfilerReport-1617070263: InProgress
.........
2021-03-30 02:12:47 Starting - Preparing the instances for training......
2021-03-30 02:13:48 Downloading - Downloading input data...
2021-03-30 02:14:34 Training - Training image download completed. Training in progress..Docker entrypoint called with argument(s): train
Running default environment configuration script
[03/30/2021 02:14:39 INFO 140206502115136] Reading default configuration from /opt/amazon/lib/python3.7/site-packages/algorithm/resources/default-input.json: {'mini_batch_size': '1000', 'epochs': '15', 'feature_dim': 'auto', 'use_bias': 'true', 'binary_classifier_model_selection_criteria': 'accuracy', 'f_beta': '1.0', 'target_recall': '0.8', 'target_precision': '0.8', 'num_models': 'auto', 'num_calibration_samples': '10000000', 'init_method': 'uniform', 'init_scale': '0.07', 'init_sigma': '0.01', 'init_b

[2021-03-30 02:14:55.136] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 5, "duration": 7703, "num_examples": 200, "num_bytes": 33493320}
#metrics {"StartTime": 1617070495.1365197, "EndTime": 1617070495.1367676, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 1, "model": 0}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.44001085551659663, "count": 1, "min": 0.44001085551659663, "max": 0.44001085551659663}}}

#metrics {"StartTime": 1617070495.1368644, "EndTime": 1617070495.136883, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 1, "model": 1}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.4105252083629819, "count": 1, "min": 0.4105252083629819, "max": 0.4105252083629819}}}

#metrics {"StartTime": 1617070495.1369395, "EndTime": 1617070495.13695, "Dimensions": {"Algorithm": 

[2021-03-30 02:15:02.265] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 7, "duration": 7115, "num_examples": 200, "num_bytes": 33493320}
#metrics {"StartTime": 1617070502.2651634, "EndTime": 1617070502.26524, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 2, "model": 0}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.3963692446761395, "count": 1, "min": 0.3963692446761395, "max": 0.3963692446761395}}}

#metrics {"StartTime": 1617070502.2653294, "EndTime": 1617070502.2653503, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 2, "model": 1}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.37658528819635284, "count": 1, "min": 0.37658528819635284, "max": 0.37658528819635284}}}

#metrics {"StartTime": 1617070502.2654037, "EndTime": 1617070502.265419, "Dimensions": {"Algorithm": 

[2021-03-30 02:15:10.176] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 9, "duration": 7899, "num_examples": 200, "num_bytes": 33493320}
#metrics {"StartTime": 1617070510.1761956, "EndTime": 1617070510.1764195, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 3, "model": 0}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.3764211316132665, "count": 1, "min": 0.3764211316132665, "max": 0.3764211316132665}}}

#metrics {"StartTime": 1617070510.176511, "EndTime": 1617070510.1765296, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 3, "model": 1}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.36157515816233265, "count": 1, "min": 0.36157515816233265, "max": 0.36157515816233265}}}

#metrics {"StartTime": 1617070510.1765773, "EndTime": 1617070510.1765878, "Dimensions": {"Algorithm"

...

[2021-03-30 02:15:52.238] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 21, "duration": 6729, "num_examples": 200, "num_bytes": 33493320}
#metrics {"StartTime": 1617070552.2383473, "EndTime": 1617070552.238431, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 9, "model": 0}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.34497470242294237, "count": 1, "min": 0.34497470242294237, "max": 0.34497470242294237}}}

#metrics {"StartTime": 1617070552.2385654, "EndTime": 1617070552.2385876, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 9, "model": 1}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.33982735768993894, "count": 1, "min": 0.33982735768993894, "max": 0.33982735768993894}}}

#metrics {"StartTime": 1617070552.2387025, "EndTime": 1617070552.2387218, "Dimensions": {"Algori

[2021-03-30 02:16:07.578] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 25, "duration": 7911, "num_examples": 200, "num_bytes": 33493320}
#metrics {"StartTime": 1617070567.5788052, "EndTime": 1617070567.578902, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 11, "model": 0}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.3421441708282011, "count": 1, "min": 0.3421441708282011, "max": 0.3421441708282011}}}

#metrics {"StartTime": 1617070567.5789788, "EndTime": 1617070567.5789988, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 11, "model": 1}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.3377189590607456, "count": 1, "min": 0.3377189590607456, "max": 0.3377189590607456}}}

#metrics {"StartTime": 1617070567.5790565, "EndTime": 1617070567.5790732, "Dimensions": {"Algorithm"

[2021-03-30 02:16:14.675] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 27, "duration": 7085, "num_examples": 200, "num_bytes": 33493320}
#metrics {"StartTime": 1617070574.675322, "EndTime": 1617070574.675409, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 12, "model": 0}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.3411415676327806, "count": 1, "min": 0.3411415676327806, "max": 0.3411415676327806}}}

#metrics {"StartTime": 1617070574.6754928, "EndTime": 1617070574.6755142, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 12, "model": 1}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.3368815051898285, "count": 1, "min": 0.3368815051898285, "max": 0.3368815051898285}}}

#metrics {"StartTime": 1617070574.6755674, "EndTime": 1617070574.6755843, "Dimensions": {"Algorithm":


2021-03-30 02:16:32 Uploading - Uploading generated training model[2021-03-30 02:16:22.067] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 29, "duration": 7379, "num_examples": 200, "num_bytes": 33493320}
#metrics {"StartTime": 1617070582.0677953, "EndTime": 1617070582.0678809, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 13, "model": 0}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.3403380544844584, "count": 1, "min": 0.3403380544844584, "max": 0.3403380544844584}}}

#metrics {"StartTime": 1617070582.0679586, "EndTime": 1617070582.067979, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 13, "model": 1}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.3361356634782187, "count": 1, "min": 0.3361356634782187, "max": 0.3361356634782187}}}

#metrics {"StartTime": 1617070582


2021-03-30 02:16:48 Completed - Training job completed
Training seconds: 184
Billable seconds: 184
CPU times: user 932 ms, sys: 34.2 ms, total: 967 ms
Wall time: 6min 14s


In [48]:
%%time 
# deploy and create a predictor
modeldesign_predictor = linear_modeldesign.deploy(initial_instance_count=1, instance_type='ml.t2.medium')

Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.


-----------------!CPU times: user 290 ms, sys: 21.5 ms, total: 312 ms
Wall time: 8min 32s


In [49]:
print('Metrics for designed, LinearLearner.\n')

# get metrics for balanced predictor
metrics = evaluate(modeldesign_predictor, 
                   test_features.to_numpy(dtype=np.float32), 
                   test_labels, 
                   verbose=True)

Metrics for designed, LinearLearner.

prediction (col)    0.0  1.0
actual (row)                
0                 85275   26
1                    31  110

Recall:     0.780
Precision:  0.809
Accuracy:   0.999



In [50]:
## IMPORTANT
# delete the predictor endpoint after evaluation 
# delete the predictor endpoint 
delete_endpoint(balanced_predictor)

The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


Already deleted: linear-learner-2021-03-30-01-50-50-827


## Final Cleanup!

* Double check that you have deleted all your endpoints.
* I'd also suggest manually deleting your S3 bucket, models, and endpoint configurations directly from your AWS console.

You can find thorough cleanup instructions, [in the documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/ex1-cleanup.html).

---
# Conclusion

In this notebook, you saw how to train and deploy a LinearLearner in SageMaker. This model is well-suited for a binary classification task that involves specific design decisions and managing class imbalance in the training set.

Following the steps of a machine learning workflow, you loaded in some credit card transaction data, explored that data and prepared it for model training. Then trained, deployed, and evaluated several models, according to different design considerations!